# Công cụ quản lý Kubernetes Cluster trên giao diện Web - Kubernetes Dashboard

In [2]:
!curl https://raw.githubusercontent.com/kubernetes/dashboard/v2.7.0/aio/deploy/recommended.yaml > dashboard-v2-7.yaml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  7621  100  7621    0     0   8314      0 --:--:-- --:--:-- --:--:--  8328


In [7]:
# Triển khai
!kubectl apply -f dashboard-v2-7.yaml

namespace/kubernetes-dashboard unchanged
serviceaccount/kubernetes-dashboard unchanged
service/kubernetes-dashboard unchanged
secret/kubernetes-dashboard-csrf unchanged
secret/kubernetes-dashboard-key-holder unchanged
configmap/kubernetes-dashboard-settings unchanged
role.rbac.authorization.k8s.io/kubernetes-dashboard unchanged
clusterrole.rbac.authorization.k8s.io/kubernetes-dashboard unchanged
rolebinding.rbac.authorization.k8s.io/kubernetes-dashboard unchanged
clusterrolebinding.rbac.authorization.k8s.io/kubernetes-dashboard unchanged
deployment.apps/kubernetes-dashboard unchanged
service/dashboard-metrics-scraper unchanged
deployment.apps/dashboard-metrics-scraper unchanged


In [26]:
!kubectl get po -n kubernetes-dashboard

NAME                                         READY   STATUS    RESTARTS   AGE
dashboard-metrics-scraper-64bcc67c9c-sb7f6   1/1     Running   0          23m
kubernetes-dashboard-5c8bd6b59-z4khp         1/1     Running   0          23m


## Tạo kubernetes-dashboard-certs, xác thực SSL

In [15]:
# Cài OpenSSL
!docker build -t myopenssl -f Dockerfile .

#1 [internal] load .dockerignore
#1 transferring context:
#1 transferring context: 2B 0.0s done
#1 DONE 0.4s

#2 [internal] load build definition from Dockerfile
#2 transferring dockerfile: 174B 0.0s done
#2 DONE 0.5s

#3 [internal] load metadata for docker.io/library/alpine:latest
#3 ...

#4 [auth] library/alpine:pull token for registry-1.docker.io
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/alpine:latest
#3 DONE 4.4s

#5 [1/3] FROM docker.io/library/alpine@sha256:124c7d2707904eea7431fffe91522a01e5a861a624ee31d03372cc1d138a3126
#5 resolve docker.io/library/alpine@sha256:124c7d2707904eea7431fffe91522a01e5a861a624ee31d03372cc1d138a3126
#5 resolve docker.io/library/alpine@sha256:124c7d2707904eea7431fffe91522a01e5a861a624ee31d03372cc1d138a3126 0.1s done
#5 sha256:124c7d2707904eea7431fffe91522a01e5a861a624ee31d03372cc1d138a3126 1.64kB / 1.64kB done
#5 sha256:b6ca290b6b4cdcca5b3db3ffa338ee0285c11744b4a6abaa9627746ee3291d8d 528B / 528B done
#5 sha256:9ed4aefc74f6792b5a804

In [ ]:
!mkdir certs
!docker run --rm -v ${PWD}/certs:/certs/ myopenssl req -nodes -newkey rsa:2048 -keyout /certs/dashboard.key -out /certs/dashboard.csr -subj "/C=/ST=/L=/O=/OU=/CN=kubernetes-dashboard"
!docker run --rm -v ${PWD}/certs:/certs/ myopenssl x509 -req -sha256 -days 365 -in /certs/dashboard.csr -signkey /certs/dashboard.key -out /certs/dashboard.crt

In [13]:
!mkdir certs
!chmod 777 -R certs
!openssl req -nodes -newkey rsa:2048 -keyout certs/dashboard.key -out certs/dashboard.csr -subj "/C=/ST=/L=/O=/OU=/CN=kubernetes-dashboard"
!openssl x509 -req -sha256 -days 365 -in certs/dashboard.csr -signkey certs/dashboard.key -out certs/dashboard.crt
!sudo chmod -R 777 certs

In [22]:
# tạo Secret
!kubectl create secret generic kubernetes-dashboard-certs --from-file=certs -n kubernetes-dashboard

secret/kubernetes-dashboard-certs created


In [42]:
!kubectl get secret -n kubernetes-dashboard

NAME                              TYPE     DATA   AGE
kubernetes-dashboard-certs        Opaque   3      14m
kubernetes-dashboard-csrf         Opaque   1      34m
kubernetes-dashboard-key-holder   Opaque   2      34m


In [27]:
!kubectl get node

NAME             STATUS   ROLES           AGE   VERSION
docker-desktop   Ready    control-plane   37h   v1.25.9


In [28]:
!!kubectl get node -o wide

['NAME             STATUS   ROLES           AGE   VERSION   INTERNAL-IP    EXTERNAL-IP   OS-IMAGE         KERNEL-VERSION                      CONTAINER-RUNTIME',
 'docker-desktop   Ready    control-plane   37h   v1.25.9   192.168.65.4   <none>        Docker Desktop   5.15.90.1-microsoft-standard-WSL2   docker://23.0.5']

In [30]:
# truy cập địa chỉ https://172.16.10.100:31000 sẽ vào màn hình đăng nhập

In [38]:
# Lấy Token đăng nhập Dashboard: admin-user.yaml
!kubectl apply -f admin-user.yaml

serviceaccount/admin-user unchanged
clusterrolebinding.rbac.authorization.k8s.io/admin-user unchanged


In [ ]:
!kubectl -n kubernetes-dashboard describe secret $(kubectl -n kubernetes-dashboard get secret | grep admin-user | awk '{print $1}')

In [46]:
!kubectl get secret -n kubernetes-dashboard

NAME                              TYPE     DATA   AGE
kubernetes-dashboard-certs        Opaque   3      18m
kubernetes-dashboard-csrf         Opaque   1      37m
kubernetes-dashboard-key-holder   Opaque   2      37m


In [ ]:
!kubectl describe secret/admin-user-token -n kubernetes-dashboard

## Dashboard với Kubernetes trong Docker Desktop

In [ ]:
!kubectl apply -f https://raw.githubusercontent.com/kubernetes/dashboard/v2.7.0/aio/deploy/recommended.yaml

In [ ]:
!kubectl proxy

In [ ]:
#Truy cập địa chỉ sau để đăng nhập: http://localhost:8001/api/v1/namespaces/kubernetes-dashboard/services/https:kubernetes-dashboard:/proxy/

## Triển khai metrics server
 - metrics server trong kubernetes (metrics server) giám sát về tài nguyên sử dụng trên cluster, cung cấp các API để các thành phần khác truy vấn đến biết được và mức độ sử dụng tài nguyên (CPU, Memory) của Pod, Node ... Cần có Metric Server để HPA hoạt động chính xác

In [47]:
!kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/download/v0.6.3/components.yaml

serviceaccount/metrics-server created
clusterrole.rbac.authorization.k8s.io/system:aggregated-metrics-reader created
clusterrole.rbac.authorization.k8s.io/system:metrics-server created
rolebinding.rbac.authorization.k8s.io/metrics-server-auth-reader created
clusterrolebinding.rbac.authorization.k8s.io/metrics-server:system:auth-delegator created
clusterrolebinding.rbac.authorization.k8s.io/system:metrics-server created
service/metrics-server created
deployment.apps/metrics-server created
apiservice.apiregistration.k8s.io/v1beta1.metrics.k8s.io created


In [ ]:
!kubectl top no
!kubectl top pod